## ANLP Assignment 1: Sentiment Analysis
### Christopher Hamilton,  a1766121

In [1]:
import json
import os 

import pandas as pd
import numpy as np
from langdetect import detect as detect_language

### 1. Reading dataset and initial pre-processing

In [2]:
def read_json_to_df(file_name):
    data = []
    with open(file_name) as data_file:
        for line in data_file:
            data.append(json.loads(line))    

    return pd.json_normalize(data)

train_df = read_json_to_df("hotel_reviews_train.json")

In [3]:
train_df.head()

,title,text,date_stayed,offering_id,num_helpful_votes,date,id,via_mobile,ratings.service,ratings.cleanliness,...,ratings.rooms,author.username,author.num_reviews,author.id,author.location,author.num_cities,author.num_helpful_votes,author.num_type_reviews,ratings.check_in_front_desk,ratings.business_service_(e_g_internet_access)
0,“Horrible experience”,First of all we got there and they didn't have...,September 2012,80138,0,"September 19, 2012",140716137,False,1.0,2.0,...,1.0,Kh3RD,1.0,AB404BB664D653ECF79DE0E0867F6D34,"Las Vegas, Nevada",NaN,NaN,NaN,NaN,NaN
1,“Stay Away”,Found Racist graffiti in the room. Request to ...,June 2011,240151,1,"June 27, 2011",114807323,False,1.0,1.0,...,NaN,TheUglyPhotographer,4.0,BB116F87FE8F9AB356F63853BFD32FFE,"Oceanside, California",3.0,4.0,4.0,NaN,NaN
2,“Great location and service”,Close to Union Square - hotel is a bit of a ma...,October 2010,80793,0,"October 25, 2010",84805430,False,4.0,5.0,...,4.0,Moonstonemoclips,48.0,F3D0CF371B788300E73A1413B2DABB4B,Kirkland,31.0,27.0,32.0,NaN,NaN
3,“I will never go back here again!”,I had a bad vibe about this place from the mom...,June 2012,111418,1,"June 28, 2012",132971117,False,3.0,2.0,...,1.0,JoanellenJ,22.0,BC6BC07F81B768F78B6CE17A18762C11,New York,10.0,9.0,5.0,NaN,NaN
4,“Ripped off our VISA card after departure”,"After we received our ""final"" bill and left th...",January 2012,671150,3,"February 4, 2012",124104157,False,NaN,NaN,...,NaN,Lynnworks,3.0,F7E9D044FA2554FD06A871289312E043,Providence,3.0,7.0,3.0,NaN,NaN


In [4]:
predict_overall_train_df = train_df[["title", "text", "ratings.overall"]]

predict_overall_train_df.head()
predict_overall_train_df["ratings.overall"].value_counts()

ratings.overall
5.0    9825
4.0    7720
3.0    3287
2.0    1611
1.0    1557
Name: count, dtype: int64

In [5]:
predict_overall_train_df.loc[:,'title'] = ["".join(filter(lambda c: str.isalnum(c) or str.isspace(c), char)) for char in predict_overall_train_df.title]
predict_overall_train_df.loc[:,'text'] = ["".join(filter(lambda c: str.isalnum(c) or str.isspace(c), char)) for char in predict_overall_train_df.text]

In [6]:
predict_overall_train_df.head()

,title,text,ratings.overall
0,Horrible experience,First of all we got there and they didnt have ...,1.0
1,Stay Away,Found Racist graffiti in the room Request to c...,1.0
2,Great location and service,Close to Union Square hotel is a bit of a maz...,4.0
3,I will never go back here again,I had a bad vibe about this place from the mom...,2.0
4,Ripped off our VISA card after departure,After we received our final bill and left the ...,1.0


In [ ]:
def filter_english_reviews(df):
    try:
        title_languages = detect_language(df["title"].tolist())
        text_languages = detect_language(df["text"].tolist())
        return df[(title_languages == "en") & (text_languages == "en")]
    except Exception as e:
        print(f"Error during language detection: {e}")
        return df[[]]  # Return an empty DataFrame if detection fails

In [ ]:
predict_overall_train_df = filter_english_reviews(predict_overall_train_df)

In [ ]:
predict_overall_train_df.head()

In [ ]:
print(train_df.info())
print(predict_overall_train_df.info())

### 2. Exploratory Data Analysis (EDA)

In [ ]:
import nltk
nltk.download('punkt_tab')
nltk.download('stopwords')

In [ ]:
import matplotlib.pyplot as plt

# Plot distribution of ratings
train_df['ratings.overall'].value_counts().sort_index().plot(kind='bar', figsize=(8,5), color='skyblue')

plt.xlabel("Rating")
plt.ylabel("Count")
plt.title("Distribution of Ratings")
plt.show()

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(stop_words="english", max_features=5000)  # Remove common words
X = vectorizer.fit_transform(train_df["text"])

# Convert to DataFrame
tfidf_df = pd.DataFrame(X.toarray(), columns=vectorizer.get_feature_names_out())

correlations = tfidf_df.corrwith(train_df["ratings.overall"])
correlations = correlations.sort_values(ascending=False)  # Sort by importance

# Find words with the weakest correlation (near zero)
non_predictive_words = correlations.sort_values(key=lambda x: np.abs(x))
print("Non-Predictive Words:\n", non_predictive_words.head(10))

# Display top positive and negative correlated words
print("Most Positive Words:\n", correlations.head(10))
print("\nMost Negative Words:\n", correlations.tail(10))

In [ ]:
# Split all reviews into words and find unique ones
all_words_text = predict_overall_train_df.text.apply(nltk.word_tokenize)
all_words_title = predict_overall_train_df.title.apply(nltk.word_tokenize)

all_words = pd.concat([all_words_text, all_words_title])
unique_words = np.unique(all_words)

print("Total Unique Words:", len(unique_words))


In [ ]:
# Word frequency plots are used to visualize the most common words in a dataset.

from collections import Counter
import matplotlib.pyplot as plt
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

stop_words = set(stopwords.words('english'))

tokens = [
    word.lower() for sentence in unique_words.tolist()
    for s in sentence
    for word in word_tokenize(s)
    if word.isalnum() and word.lower() not in stop_words
]

word_freq = Counter(tokens)

plt.figure(figsize=(12, 5))
plt.bar(*zip(*word_freq.most_common(20)))
plt.xlabel("Words")
plt.ylabel("Frequency")
plt.title("Word Frequency Plot")
plt.show()


In [ ]:
# Identifying common N-grams: N-grams are sequences of words. 
# Identifying common n-grams can give insight into the most common phrases in the dataset.

import nltk
from nltk import ngrams
from collections import Counter
import matplotlib.pyplot as plt

# Function to generate n-grams
def generate_ngrams(text, n):
    n_grams = ngrams(text, n)
    return [' '.join(gram) for gram in n_grams]

# Specify the value of n for n-grams
n_value = 3  # You can change this value to see different n-grams, e.g., 2 for bigrams, 3 for trigrams, etc.

# Generate n-grams
ngrams_list = generate_ngrams(tokens, n_value)

# Count the occurrences of each n-gram
ngrams_count = Counter(ngrams_list)

# Display the distribution
print(f"Distribution of {n_value}-grams:")
for ngram, count in ngrams_count.items():
    print(f"{ngram}: {count}")

# Plot the distribution
labels, values = zip(*ngrams_count.items())
indexes = range(len(labels))

plt.bar(indexes, values)
plt.xlabel(f'{n_value}-grams')
plt.ylabel('Frequency')
plt.xticks(indexes, labels, rotation='vertical')
plt.title(f'Distribution of {n_value}-grams')
plt.show()

### 3. Selection and training Machine Learning models

In [7]:
from sklearn.model_selection import train_test_split

X_res = pd.concat([predict_overall_train_df["title"], predict_overall_train_df["text"]], axis=1)
X_res = X_res["title"] + " " + X_res["text"]
y_res = predict_overall_train_df["ratings.overall"]

X_train, X_test, y_train, y_test = train_test_split(X_res, y_res, test_size=0.2, shuffle=True)


In [8]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer()
X_train_vectors = vectorizer.fit_transform(X_train)
X_test_vectors = vectorizer.transform(X_test)

In [9]:
result_dict_train = {}
result_dict_test = {}


In [ ]:
from sklearn.model_selection import cross_val_score


#### Multinomial Naive Bayes

In [ ]:
from sklearn.naive_bayes import MultinomialNB

# https://www.geeksforgeeks.org/multinomial-naive-bayes/

classifier = MultinomialNB()

In [ ]:
nb_accuracies = cross_val_score(classifier, X_train_vectors, y_train, cv=5)
classifier.fit(X_train_vectors, y_train)
print("Naive Bayes Train Score:", np.mean(nb_accuracies))
result_dict_train["Naive Bayes Default Train Score"] = np.mean(nb_accuracies)

In [ ]:
naive_bayes_score = classifier.score(X_test_vectors, y_test)
print("Naive Bayes Test Score:", naive_bayes_score)
result_dict_test["Naive Bayes Default Test Score"] = naive_bayes_score


#### SVM

In [ ]:
from sklearn.svm import SVC

classifier = SVC()

In [ ]:
svc_accuracies = cross_val_score(classifier, X_train_vectors, y_train, cv=5)
classifier.fit(X_train_vectors, y_train)
print("SVM Train Score:", np.mean(svc_accuracies))
result_dict_train["SVM Default Train Score"] = np.mean(svc_accuracies)

In [ ]:
svc_score = classifier.score(X_test_vectors, y_test)
print("SVM Test Score:", svc_score)
result_dict_test["SVM Default Test Score"] = svc_score


In [10]:
import tensorflow as tf

# Limit GPU memory usage
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.set_logical_device_configuration(
                gpu,
                [tf.config.LogicalDeviceConfiguration(memory_limit=(6 * 1024))])
        logical_gpus = tf.config.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        print(e)

2025-03-28 18:11:13.891310: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-03-28 18:11:13.900671: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1743147673.910271   12107 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1743147673.912904   12107 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1743147673.919816   12107 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking 

1 Physical GPUs, 1 Logical GPUs


I0000 00:00:1743147675.262086   12107 gpu_device.cc:2019] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 6144 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3050, pci bus id: 0000:01:00.0, compute capability: 8.6


In [ ]:
subset_size = 5000 # X_train_vectors.shape[0]

from tensorflow.keras.utils import to_categorical
y_train_adjusted = y_train - 1
y_test_adjusted = y_test - 1  

y_train_cat = to_categorical(y_train_adjusted, num_classes=5)
y_test_cat = to_categorical(y_test_adjusted, num_classes=5)

X_train_subset = X_train_vectors[:subset_size]
y_train_subset = y_train_cat[:subset_size]
X_val_subset = X_test_vectors[:subset_size]
y_val_subset = y_test_cat[:subset_size]

X_train_tfidf = X_train_subset.toarray()
X_val_tfidf = X_val_subset.toarray()

# Define the model
model = tf.keras.Sequential([
    tf.keras.layers.InputLayer(input_shape=(X_train_tfidf.shape[1],)),  # Input layer
    tf.keras.layers.Dense(128, activation='relu'),  # Hidden layer
    tf.keras.layers.Dropout(0.5),  # Dropout for regularization
    tf.keras.layers.Dense(5, activation='softmax')  # Output layer for 5 classes
])

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Create TensorFlow datasets
train_dataset = tf.data.Dataset.from_tensor_slices((X_train_tfidf, y_train_subset)).batch(8).prefetch(tf.data.AUTOTUNE)
val_dataset = tf.data.Dataset.from_tensor_slices((X_val_tfidf, y_val_subset)).batch(8).prefetch(tf.data.AUTOTUNE)

# Train the model
history = model.fit(train_dataset, epochs=10, validation_data=val_dataset)

/home/dev/repos/COMP-SCI-7417-Applied-Natural-Language-Processing/.venv/lib/python3.12/site-packages/keras/src/layers/core/input_layer.py:27: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead.
  warnings.warn(


Epoch 1/10


I0000 00:00:1743147680.644927   12200 service.cc:152] XLA service 0x7820fc0029e0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1743147680.644942   12200 service.cc:160]   StreamExecutor device (0): NVIDIA GeForce RTX 3050, Compute Capability 8.6
2025-03-28 18:11:20.659046: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1743147680.721697   12200 cuda_dnn.cc:529] Loaded cuDNN version 90300


 43/625 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 0.3639 - loss: 1.5294

I0000 00:00:1743147681.725449   12200 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


625/625 ━━━━━━━━━━━━━━━━━━━━ 6s 6ms/step - accuracy: 0.4770 - loss: 1.2252 - val_accuracy: 0.5962 - val_loss: 0.9336
Epoch 2/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step - accuracy: 0.6856 - loss: 0.7768 - val_accuracy: 0.6037 - val_loss: 0.9476
Epoch 3/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step - accuracy: 0.7841 - loss: 0.5499 - val_accuracy: 0.5975 - val_loss: 0.9676
Epoch 4/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step - accuracy: 0.8727 - loss: 0.3721 - val_accuracy: 0.5885 - val_loss: 1.0662
Epoch 5/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 4s 6ms/step - accuracy: 0.9182 - loss: 0.2538 - val_accuracy: 0.5927 - val_loss: 1.0762
Epoch 6/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step - accuracy: 0.9466 - loss: 0.1871 - val_accuracy: 0.5879 - val_loss: 1.2293
Epoch 7/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step - accuracy: 0.9580 - loss: 0.1472 - val_accuracy: 0.5750 - val_loss: 1.2839
Epoch 8/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step - accuracy: 0.9692 - loss: 0.1101 - val_accuracy: 0.5821 - val_

In [13]:
# Convert sparse matrices to dense matrices
X_train_tfidf = X_train_vectors.toarray()
X_val_tfidf = X_test_vectors.toarray()

# Create TensorFlow datasets
train_dataset = tf.data.Dataset.from_tensor_slices((X_train_tfidf, y_train)).batch(8).prefetch(tf.data.AUTOTUNE)
val_dataset = tf.data.Dataset.from_tensor_slices((X_val_tfidf, y_test)).batch(8).prefetch(tf.data.AUTOTUNE)
# Train the model
history = model.fit(train_dataset, epochs=10, validation_data=val_dataset)


2025-03-28 17:38:29.672884: W external/local_xla/xla/tsl/framework/bfc_allocator.cc:501] Allocator (GPU_0_bfc) ran out of memory trying to allocate 6.76GiB (rounded to 7263744000)requested by op _EagerConst
If the cause is memory fragmentation maybe the environment variable 'TF_GPU_ALLOCATOR=cuda_malloc_async' will improve the situation. 
Current allocation summary follows.
Current allocation summary follows.
2025-03-28 17:38:29.672919: I external/local_xla/xla/tsl/framework/bfc_allocator.cc:1058] BFCAllocator dump for GPU_0_bfc
2025-03-28 17:38:29.672930: I external/local_xla/xla/tsl/framework/bfc_allocator.cc:1065] Bin (256): 	Total Chunks: 18, Chunks in use: 18. 4.5KiB allocated for chunks. 4.5KiB in use in bin. 608B client-requested in use in bin.
2025-03-28 17:38:29.672937: I external/local_xla/xla/tsl/framework/bfc_allocator.cc:1065] Bin (512): 	Total Chunks: 1, Chunks in use: 1. 512B allocated for chunks. 512B in use in bin. 512B client-requested in use in bin.
2025-03-28 17:38:

InternalError: Failed copying input tensor from /job:localhost/replica:0/task:0/device:CPU:0 to /job:localhost/replica:0/task:0/device:GPU:0 in order to run _EagerConst: Dst tensor is not initialized.

#### LSTM

In [ ]:
from tensorflow.keras import layers, optimizers, models

vocabulary_size = len(tokenizer.word_counts)

def create_rnn_lstm():
    model = layers.Sequential()
    model.add(layers.Embedding(vocabulary_size, 25, input_length=seq_len))
    model.add(layers.LSTM(150, return_sequences=True))
    model.add(layers.LSTM(150))
    model.add(layers.Dense(150, activation='relu'))

    model.add(layers.Dense(vocabulary_size, activation='softmax'))
    
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
   
    model.summary()
    
    return model

classifier = create_rnn_lstm()

classifier.fit(X_train_vectors, y_train, epochs=5)

In [ ]:
print(result_dict_train)
print(result_dict_test)

### 4. Experiment with VADER sentiment lexicon

### 5. Final testing on test set and discussion of results

### 6. Propose a method to predict aspects 

***(COMP SCI 7417 and COMP SCI 7717 only)***

### 7. Reflection on the ***Product*** development.

### 9. References

https://pandas.pydata.org/pandas-docs/version/1.2.0/reference/api/pandas.json_normalize.html

https://www.analyticsvidhya.com/blog/2022/07/step-by-step-exploratory-data-analysis-eda-using-python/

https://pawarbi.github.io/blog/pandas/numpy/data-cleaning/2021/03/05/removing-non-alphanumeric-symbols-characters-from-column-numpy-pandas-dataframe.html


### Appendix